# pH-dependence on added contaminants in water

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Feb 8th, 2022</p>

This tutorial demonstrates how pH is dependent on the added contaminant in the water, affecting the fish life as well
as the general ecosystem.

Below, we initialize the chemical system with a thermodynamic database and aqueous phase (governed
by the the HKF activity model). We initialize the chemical state corresponding to the pure water
(with pH approximate to 7).

In [ ]:
from reaktoro import *

# Define the thermodynamic database
db =  SupcrtDatabase("supcrt98")
# Define the aqueous phase
aqueousphase = AqueousPhase(speciate("H O Na Cl N"))
aqueousphase.setActivityModel(ActivityModelHKF())
# Define the chemical system:
system = ChemicalSystem(db, aqueousphase)

T = 25.0 # in celsius
P = 1.0 # in bar

# Define chemical state corresponding to the pure water
state = ChemicalState(system)
state.setTemperature(T, "celsius")
state.setPressure(P, "bar")
state.set("H2O(aq)", 1.0, "kg")

# Define equilibrium solver and solve equilibrium problem with initial chemical state
solver = EquilibriumSolver(system)
solver.solve(state)

# Define aqueous properties of the chemical state
aprops = AqueousProps(state)
print("pH of pure water = ", aprops.pH())

### Decreasing pH

First, we investigate the behavior of the pH when adding the acidic contaminant to the water.
We define the auxiliary lists with amounts of acid HCl in the chemical state `hcl_amounts` and the list of pH `phs`
(corresponding to the added acid amount). Both lists are populated in the loop of 50 steps. We gradually add 0.1 mmol
of hydrogen chloride and evaluate the pH in the obtained state.

In [ ]:
# Initialize lists with HCl amounts and ph values
hcl_initial = 0.0
hcl_delta = 0.1
nsteps = 50
hcl_amounts = [hcl_initial]
phs = [aprops.pH()[0]]

# Run loop of nsteps steps
for i in range(nsteps):

    # Add more hydrogen chlorite to the problem
    state.add("HCl(aq)", hcl_delta, "mmol")

    # Equilibrate state with updated problem
    solver.solve(state)

    # Update aqueous properties
    aprops.update(state)

    # Append new ph
    phs.append(aprops.pH()[0])

    # Append new hydrogen chlorite amount
    hcl_amounts.append(hcl_amounts[-1] + hcl_delta)

### Increasing pH

If we add in a chemical contaminant such as ammonia (a compound of nitrogen and hydrogen with the formula NH3,
colorless gas with a characteristic pungent smell), that can increase the pH and affect fish life.

In [ ]:
state = ChemicalState(system)
state.setTemperature(T, "celsius")
state.setPressure(P, "bar")
state.set("H2O(aq)", 1.0, "kg")
solver.solve(state)
aprops.update(state)

nh3_initial = 0.0
nh3_amounts = [nh3_initial]
phs_increase = [aprops.pH()[0]]

nh3_delta = 0.1
nsteps = 50

for i in range(nsteps):

    # Add more ammonia to the problem
    state.add("NH3(aq)", nh3_delta, "mmol")

    # Equilibrate state with updated problem
    solver.solve(state)

    # Update aqueous properties
    aprops.update(state)

    # Append new ph
    phs_increase.append(aprops.pH()[0])

    # Append new ammonia amount
    nh3_amounts.append(nh3_amounts[-1] + nh3_delta)

### Plotting the pH dependencies

Let us plot pH as a function of the added HCl and NH<sup>3</sup>(aq) amounts:

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1)

ax1.plot(hcl_amounts, phs, label=f"pH", color='C3')
ax1.legend(loc="best")
ax1.set_title('Dependence of pH on HCl amount')
ax1.grid(True)
ax1.set_ylabel('pH [-]')
ax1.set_xlabel(r'HCl amount [mol]')

ax2.plot(nh3_amounts, phs_increase, label=f"pH", color='C4')
ax2.set_title(r'Dependence of pH on NH$\sf{_3}$ amount')
ax2.legend(loc="best")
ax2.grid(True)
ax2.set_ylabel('pH [-]')
ax2.set_xlabel(r'NH$\sf{_3}$ amount [mol]')

fig.tight_layout()

As expected, adding the contaminant like HCl into water increases free H<sup>+</sup> ions and decreases pH according to the reaction
$$
\sf{HCl(aq) + H_2O(l) = H_3O^+ + Cl^-},
$$
affecting the ecosystem, whereas adding ammonia removes H<sup>+</sup> from water to produce ammonium and hydroxide, i.e.,
to produce ammonium and hydroxide:
$$
\sf{NH_3 + H_2O = NH_4^+ + OH^-}.
$$